In [ ]:
import kagglehub

path = kagglehub.dataset_download("muhammadimran112233/employees-evaluation-for-promotion")

print("Path to dataset files:", path)

100%|██████████| 640k/640k [00:00<00:00, 20.8MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/muhammadimran112233/employees-evaluation-for-promotion/versions/1


In [ ]:
import os
print(os.listdir(path))


['employee_promotion.csv']


In [ ]:
import pandas as pd

In [ ]:
import os
df = pd.read_csv(os.path.join(path, 'employee_promotion.csv'))

In [ ]:
df.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,0,49.0,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,60.0,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,50.0,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,50.0,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,73.0,0


In [ ]:
df['previous_year_rating'].unique()

array([ 5.,  3.,  1.,  4., nan,  2.])

In [ ]:
df.columns

Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'awards_won', 'avg_training_score', 'is_promoted'],
      dtype='object')

In [ ]:
df.isnull().sum()

,0
employee_id,0
department,0
region,0
education,2409
gender,0
recruitment_channel,0
no_of_trainings,0
age,0
previous_year_rating,4124
length_of_service,0


In [ ]:
df_prev_year=df.loc[df['previous_year_rating'].isnull(),'length_of_service']

In [ ]:
df['previous_year_rating']=df['previous_year_rating'].fillna(-1)

In [ ]:
df_avg=df.loc[df['avg_training_score'].isnull(),'previous_year_rating']

In [ ]:
df_avg.value_counts(normalize=True)

,proportion
previous_year_rating,
3.0,0.340234
5.0,0.232031
4.0,0.187109
1.0,0.096094
-1.0,0.078906
2.0,0.065625


In [ ]:
df_prev_year.unique()

array([1])

In [ ]:
df.columns

Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'awards_won', 'avg_training_score', 'is_promoted'],
      dtype='object')

In [ ]:
df['region'].value_counts()

,count
region,
region_2,12343
region_22,6428
region_7,4843
region_15,2808
region_13,2648
region_26,2260
region_31,1935
region_4,1703
region_27,1659


In [ ]:
print(df.groupby('is_promoted')['gender'].value_counts(normalize=True))

is_promoted  gender
0            m         0.703929
             f         0.296071
1            m         0.685733
             f         0.314267
Name: proportion, dtype: float64


In [ ]:
df=df.drop(columns=['employee_id','region','education','gender','recruitment_channel','age','avg_training_score'])

In [ ]:
df.isnull().sum()

,0
department,0
no_of_trainings,0
previous_year_rating,0
length_of_service,0
awards_won,0
is_promoted,0


In [ ]:
df['is_promoted'].value_counts(normalize=True)

,proportion
is_promoted,
0,0.91483
1,0.08517


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import joblib

In [ ]:
df['awards_per_year'] = df['awards_won'] / (df['length_of_service'] + 1)

In [ ]:
df['training_intensity'] = df['no_of_trainings'] / (df['length_of_service'] + 1)

In [ ]:
df['performance_awards_ratio'] = df['previous_year_rating'] * df['awards_won']

In [ ]:
dept_promotion_rates = df.groupby('department')['is_promoted'].mean().to_dict()
overall_promotion_rate = df['is_promoted'].mean()

In [ ]:
dept_promotion_rates

{'Analytics': 0.09566517189835576,
 'Finance': 0.08123028391167192,
 'HR': 0.05624483043837883,
 'Legal': 0.05101058710298364,
 'Operations': 0.0901480437081424,
 'Procurement': 0.0963855421686747,
 'R&D': 0.06906906906906907,
 'Sales & Marketing': 0.07203087885985748,
 'Technology': 0.10759316335107873}

In [ ]:
joblib.dump(dept_promotion_rates, 'dept_encoding.pkl')
joblib.dump(overall_promotion_rate, 'overall_promotion_rate.pkl')

['overall_promotion_rate.pkl']

In [ ]:
df['dept_encoded'] = df['department'].map(dept_promotion_rates)

In [ ]:
feature_columns = [
    'no_of_trainings',
    'previous_year_rating',
    'length_of_service',
    'awards_won',
    'dept_encoded',
    'awards_per_year',
    'training_intensity',
    'performance_awards_ratio'
]

In [ ]:
X = df[feature_columns]
y = df['is_promoted']

In [ ]:
joblib.dump(feature_columns, 'feature_columns.pkl')

['feature_columns.pkl']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y  )

In [ ]:
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

In [ ]:
y_train_balanced.value_counts()

,count
is_promoted,
0,40112
1,40112


In [ ]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42, max_depth=10),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss')
}

In [ ]:
results = {}

In [ ]:
for name, model in models.items():
    print(f"Training: {name}")
    print('='*50)

    model.fit(X_train_balanced, y_train_balanced)
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    results[name] = {'model': model,'f1_score': f1,'roc_auc': roc_auc,'y_pred': y_pred
    }

    print(f"\nF1 Score: {f1:.4f}")
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred))
    print(f"\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

Training: Logistic Regression

F1 Score: 0.2165
ROC-AUC Score: 0.6790

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.65      0.77     10028
           1       0.13      0.58      0.22       934

    accuracy                           0.64     10962
   macro avg       0.54      0.61      0.49     10962
weighted avg       0.87      0.64      0.72     10962


Confusion Matrix:
[[6521 3507]
 [ 395  539]]
Training: Decision Tree

F1 Score: 0.2255
ROC-AUC Score: 0.6704

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.71      0.81     10028
           1       0.14      0.53      0.23       934

    accuracy                           0.69     10962
   macro avg       0.54      0.62      0.52     10962
weighted avg       0.87      0.69      0.76     10962


Confusion Matrix:
[[7083 2945]
 [ 441  493]]
Training: Random Forest

F1 Score: 0.2497
ROC-AUC Score: 0.6859

Classif

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(
        max_iter=1000,
        random_state=42,
        class_weight='balanced'
    ),
    'Decision Tree': DecisionTreeClassifier(
        random_state=42,
        max_depth=10,
        class_weight='balanced'
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=200,
        random_state=42,
        min_samples_split=50,
        min_samples_leaf=20,
        class_weight='balanced'
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=100,
        random_state=42
    ),
    'XGBoost': XGBClassifier(
        n_estimators=100,
        random_state=42,
        eval_metric='logloss',
        scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum()
    )
}

In [ ]:
from sklearn.metrics import precision_score

In [ ]:
results = {}

for name, model in models.items():
    print(f"\n{'='*50}")
    print(f"Training: {name}")
    print('='*50)

    model.fit(X_train, y_train)

    y_pred_proba = model.predict_proba(X_test)[:, 1]

    best_f1 = 0
    best_threshold = 0.5
    best_precision = 0

    for threshold in np.arange(0.3, 0.9, 0.05):
        y_pred_temp = (y_pred_proba >= threshold).astype(int)
        f1 = f1_score(y_test, y_pred_temp)
        precision = precision_score(y_test, y_pred_temp, zero_division=0)

        if f1 > best_f1 and precision >= 0.25:
            best_f1 = f1
            best_threshold = threshold
            best_precision = precision

    y_pred = (y_pred_proba >= best_threshold).astype(int)

    roc_auc = roc_auc_score(y_test, y_pred_proba)

    results[name] = {
        'model': model,
        'f1_score': best_f1,
        'roc_auc': roc_auc,
        'threshold': best_threshold,
        'precision': best_precision,
        'y_pred': y_pred
    }

    print(f"\nBest Threshold: {best_threshold:.2f}")
    print(f"F1 Score: {best_f1:.4f}")
    print(f"Precision (class 1): {best_precision:.4f}")
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred))
    print(f"\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))



Training: Logistic Regression

Best Threshold: 0.65
F1 Score: 0.2385
Precision (class 1): 0.3499
ROC-AUC Score: 0.6808

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95     10028
           1       0.35      0.18      0.24       934

    accuracy                           0.90     10962
   macro avg       0.64      0.57      0.59     10962
weighted avg       0.88      0.90      0.89     10962


Confusion Matrix:
[[9714  314]
 [ 765  169]]

Training: Decision Tree

Best Threshold: 0.70
F1 Score: 0.2787
Precision (class 1): 0.3369
ROC-AUC Score: 0.6815

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.96      0.94     10028
           1       0.34      0.24      0.28       934

    accuracy                           0.90     10962
   macro avg       0.63      0.60      0.61     10962
weighted avg       0.88      0.90      0.89     10962


Confusion Matrix:

In [ ]:
best_model_name = max(results, key=lambda x: results[x]['f1_score'])
best_model = results[best_model_name]['model']
best_threshold = results[best_model_name]['threshold']

In [ ]:
joblib.dump(best_model, 'best_promotion_model.pkl')
joblib.dump(best_threshold, 'best_threshold.pkl')

['best_threshold.pkl']

In [ ]:
# decision tree at threshold 0.7